https://www.kaggle.com/code/slashafk/getting-started-with-nlp-for-absolute-beginners/edit

https://www.kaggle.com/code/slashafk/iterate-like-a-grandmaster/edit

# Kaggle setup

In [1]:
DOWNLOAD_DEPS = True

In [ ]:
import sys

dependencies = [
    "kagtool",
    "pandas",
    "matplotlib",
    "scikit-learn",
    "kaggle",
    "fastai",
    "transformers",
    "datasets",
    "evaluate",
    "accelerate",
    "sentencepiece",
    "protobuf"
]

if DOWNLOAD_DEPS:
    !pip install -U {" ".join(dependencies)} --target whl
else:
    !pip install -U {" ".join(dependencies)}


sys.path.append("whl")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 46.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 55.1 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 47.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load or Save Model

In [13]:
from kagtool.kaggle_downloader import KaggleDownloader

dataset_name = 'us-patent-phrase-to-phrase-matching'
creds = ''
model_dir = '/kaggle/working/model'
model_nm = 'microsoft/deberta-v3-small'
num_labels = 1

dataset_path = KaggleDownloader(dataset_name, creds).load_or_fetch_kaggle_dataset()

us-patent-phrase-to-phrase-matching.zip: Skipping, found more recently modified local copy (use --force to force download)


# Deps

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# or from fastai.imports import *
# from fastai.imports import *

# Dataset & EDA

In [ ]:
df = pd.read_csv(path/'train.csv')

In [ ]:
df

In [ ]:
df.describe(include='object')

It's important to carefully read the [dataset](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data) description to understand how each of these columns is used.

In [ ]:
eval_df = pd.read_csv(path/'test.csv')
len(eval_df)

In [ ]:
eval_df.head()

Distribution of values of `target`

In [ ]:
df.target.describe()

In [ ]:
col = df.target
col.value_counts()

In [ ]:
uniqs = col.nunique() / len(col) * 100,
dups = (col.duplicated().sum() / len(df)) * 100
uniqs, dups

**What can you observe?**

Lots of target, 80% unique values, feel like X variable?

**Let's look at the values**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_text_length(df, col):
    df['text_length'] = df[col].apply(len)
    length_counts = df['text_length'].value_counts().sort_index(ascending=True)

    length_counts.plot(kind='bar', figsize=(10, 6))
    plt.xlabel('Text Length')
    plt.ylabel('Count')
    plt.title('Breakdown of Text Lengths')
    plt.show()

In [ ]:
plot_text_length(df, 'target')

In [ ]:
def display_text_examples_by_length(df, col):
    # Step 1: Find the minimum, maximum, and medium length of the text
    min_length = df[col].str.len().min()
    max_length = df[col].str.len().max()
    medium_length = df[col].str.len().median()

    # Step 2: Filter the DataFrame to include only the rows with the desired lengths
    min_length_rows = df[df[col].str.len() == min_length]
    max_length_rows = df[df[col].str.len() == max_length]
    medium_length_rows = df[df[col].str.len() == medium_length]

    # Step 3: Display the filtered DataFrame
    print("Examples of minimum length text:")
    print(min_length_rows[col].tolist()[:5])
    print("\nExamples of maximum length text:")
    print(max_length_rows[col].tolist()[:5])
    print("\nExamples of medium length text:")
    print(medium_length_rows[col].tolist()[:5])

In [ ]:
display_text_examples_by_length(df, 'target')

**Get a sense of `anchor` now**

In [ ]:
col = df.anchor

In [ ]:
col.describe()

In [ ]:
uniqs = col.nunique() / len(col) * 100,
dups = (col.duplicated().sum() / len(df)) * 100
uniqs, dups

In [ ]:
col.value_counts()

**What can you observe?**

Lots of duplicate 98%!!!

**Now let's check the values**

In [ ]:
plot_text_length(df, 'anchor')

In [ ]:
display_text_examples_by_length(df, 'anchor')

Now `context`

In [ ]:
col = df.context

In [ ]:
col.describe(include='object')

In [ ]:
col.value_counts()

In [ ]:
uniqs = col.nunique() / len(col) * 100,
dups = (col.duplicated().sum() / len(df)) * 100
uniqs, dups

**What can you observe?**

Few contexts, they feel like categories or codes

**Now, let's check the values**

In [ ]:
plot_text_length(df, 'context')

In [ ]:
display_text_examples_by_length(df, 'context')

# How can turn this problem into something I know how to work with? How can I transform the problem a bit so that I can use an angle I know -> how can I turn this into a classification pattern?

**We can transform columns into one single text column that we feed to a NLP model and classify labels**

In [ ]:
df['input'] = '#####TARGET: ' + df.target + '; #####ANCHOR: ' + df.anchor + '; #####CAT: ' + df.context

In [ ]:
df.input.head()

# Tokenization

In [ ]:
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)
ds

In [ ]:
tokz = model_loader.tokenizer

In [ ]:
def tok_func(x): return tokz(x["input"])
tok_ds = ds.map(tok_func, batched=True)

This adds a new item to our dataset called input_ids. For instance, here is the input and IDs for the first row of our data:

In [ ]:
tok_ds

In [ ]:
# this is required by huggingface for some reason
# we need to provide a column name labels so that it knows what are the Y
if 'labels' not in tok_ds.features:
  tok_ds = tok_ds.rename_columns({'score':'labels'})
tok_ds

# Validation and test sets

### Validation Set

train_test_split() divides a set in two.

We use it to create a __validation set__ that will be used to measure the model's performance

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

### Test Set

We then create a test set that is completely separate from the training process.
These are __unseen data__ that we can test the model against to see how our model perform in production

In [ ]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

# Metrics and correlation

We formatted the input for classification but we haven't defined yet what's the thing to predict.

When we are training a model, there will be one or more metrics that we want to maximize or minimize for. These are measurements that should, hopefully represent how well our model will work for us. (Don't forget to consider other things than metrics as well, real life is more than mathematical model, and it can pose ethical concerns)

For this kaggle competition, they already tell us what metrics to optimize:
Submissions are evaluated on the Pearson correlation coefficient between the predicted and actual similarity scores.


The scores are in the 0-1 range with increments of 0.25 with the following meanings:

    1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
    0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
    0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
    0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
    0.0 - Unrelated.


### Correlation - How to use?

Let's get that intuition by looking at an example. In the California Housing dataset, we want to find correlation for the median house value and other variables

In [ ]:
!pip install -U scikit-learn

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(as_frame=True).frame

In [ ]:
housing.sample(1000, random_state=52)

In [ ]:
np.set_printoptions(precision=2, suppress=True)
np.corrcoef(housing, rowvar=False)

This works well when we're getting a bunch of values at once, but it's overkill when we want a single coefficient:

In [ ]:
np.corrcoef(housing.MedInc, housing.MedHouseVal)

Therefore, we'll create this little function to just return the single number we need given a pair of variables:

In [ ]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
corr(housing.MedInc, housing.MedHouseVal)

Now we'll look at a few examples of correlations, using this function (the details of the function don't matter too much):

In [ ]:
def show_corr(df, a, b):
    x,y = df[a],df[b]
    plt.scatter(x,y, alpha=0.5, s=4)
    plt.title(f'{a} vs {b}; r: {corr(x, y):.2f}')

OK, let's check out the correlation between income and house value:

In [ ]:
show_corr(housing, 'MedInc', 'MedHouseVal')

In [ ]:
show_corr(housing, 'MedInc', 'AveRooms')

r is very sensitive to outliers. If there's outliers in your data, then the relationship between them will dominate the metric. In this case, the houses with a very high number of rooms don't tend to be that valuable, so it's decreasing r from where it would otherwise be.

Let's remove the outliers and try again:

In [ ]:
subset = housing[housing.AveRooms<15]
show_corr(subset, 'MedInc', 'AveRooms')

IN REAL LIFE YOU DONT WANT TO REMOVE OUTLIERS THOUGH, YOU NEED TO THINK ABOUT WHAT THESE OUTLIERES TELL YOU

In [ ]:
show_corr(subset, 'MedHouseVal', 'AveRooms')

In [ ]:
show_corr(subset, 'HouseAge', 'AveRooms')

# Training

Transformers expects metrics to be returned as a dict, since that way the trainer knows what label to use, so let's create a function to do that:

In [ ]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
bs = 128
epochs = 4
lr = 8e-5

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

model = model_loader.model
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
trainer.train();

The key thing to look at is the "Pearson" value in table above. As you see, it's increasing, and is already above 0.8. That's great news! It 

# Kaggle Submission

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

Look out - some of our predictions are <0, or >1! This once again shows the value of remember to actually look at your data. Let's fix those out-of-bounds predictions:

In [ ]:
preds = np.clip(preds, 0, 1)
preds

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

In [ ]:
import shutil

shutil.make_archive('debertav3_small', 'zip', save_path)